MMR (Max Marginal Relevance)?

MMR is a retrieval algorithm that tries to pick documents that are:

-> Different from each other (diverse)

-> reduces redundancy

When to use MMR in RAG?

If your chunks are very similar (policies, SOPs, safety, medical, ICSR cases):
MMR gives diverse chunks → better coverage → better grounding.

Multi-hop reasoning -> Your agent needs multiple different facts from different places.

RAG hallucination reduction->LLMs hallucinate when the retrieved documents do not cover the question -> MMR increases coverage, preventing hallucinations.




In [ ]:
from langchain.document_loaders import DirectoryLoader,TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from sentence_transformers import CrossEncoder
from langchain_chroma import Chroma
from langchain.chat_models import init_chat_model
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
from dotenv import load_dotenv

In [85]:
root_path =r"C:\Users\Mohamed Arshad\Downloads\My_RAG_Lab\llm_engineering\RAG\knowledge-base"

In [86]:
loader = DirectoryLoader(path=root_path,
                         glob="**/*.md",
                         loader_cls=TextLoader,
                         loader_kwargs={"encoding":"utf-8"})

try:
    docs =loader.load()
    print(f"Document loaded with {len(docs)} from {root_path}")

except Exception as e:
    print(f'Error occured {e}')


Document loaded with 76 from C:\Users\Mohamed Arshad\Downloads\My_RAG_Lab\llm_engineering\RAG\knowledge-base


In [87]:
docs[60]

Document(metadata={'source': 'C:\\Users\\Mohamed Arshad\\Downloads\\My_RAG_Lab\\llm_engineering\\RAG\\knowledge-base\\employees\\Oliver Spencer.md'}, page_content='# HR Record\n\n# Oliver Spencer\n\n## Summary\n- **Date of Birth**: May 14, 1990\n- **Job Title**: Backend Software Engineer\n- **Location**: Austin, Texas\n- **Current Salary**: $125,000  \n\n## Insurellm Career Progression\n- **March 2018**: Joined Insurellm as a Backend Developer I, focusing on API development for customer management systems.\n- **July 2019**: Promoted to Backend Developer II after successfully leading a team project to revamp the claims processing system, reducing response time by 30%.\n- **June 2021**: Transitioned to Backend Software Engineer with a broader role in architecture and system design, collaborating closely with the DevOps team.\n- **September 2022**: Assigned as the lead engineer for the new "Innovate" initiative, aimed at integrating AI-driven solutions into existing products.\n- **January

Initialize Hybrid Retriever ,then go for Reranker 

In [88]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunks =text_splitter.split_documents(documents=docs)

In [89]:
embedding_model =HuggingFaceEmbeddings(model="all-MiniLM-L6-V2")
dense_vector_store = Chroma.from_documents(documents=chunks,embedding=embedding_model)
# Using MMR with top 10 results and custom diversity
dense_retriever = dense_vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 10,           # Number of documents to retrieve
        "lambda_mult": 0.5 # Controls trade-off: 0=relevance only, 1=diversity only
    }
)

In [ ]:
#query ="who received the prestigious IIOTY award in 2023?"


In [172]:
query ="how many employees are there in this company"

In [173]:
retrieved_docs =dense_retreiver.invoke(query)
retrieved_docs

[Document(id='e93460ab-c195-4464-986f-79c44c4acf31', metadata={'source': 'C:\\Users\\Mohamed Arshad\\Downloads\\My_RAG_Lab\\llm_engineering\\RAG\\knowledge-base\\employees\\Rachel Martinez.md'}, page_content='- **2019:** Rating: 4.0/5\n  *Strong start as Product Manager. Successfully transitioned from APM role with minimal friction.*\n\n## Compensation History\n- **2023:** Base Salary: $135,000 + Bonus: $20,000\n- **2022:** Base Salary: $128,000 + Bonus: $15,000\n- **2021:** Base Salary: $120,000 + Bonus: $18,000\n- **2020:** Base Salary: $110,000 + Bonus: $8,000\n- **2019:** Base Salary: $105,000 + Bonus: $10,000'),
 Document(id='aeca4b05-309e-49f4-b2a7-56f0f5186af4', metadata={'source': 'C:\\Users\\Mohamed Arshad\\Downloads\\My_RAG_Lab\\llm_engineering\\RAG\\knowledge-base\\employees\\Rachel Martinez.md'}, page_content='- **2019:** Rating: 4.0/5\n  *Strong start as Product Manager. Successfully transitioned from APM role with minimal friction.*\n\n## Compensation History\n- **2023:**

In [174]:
pairs = []
for doc in retrieved_docs:
    pairs.append([query, doc.page_content])

Cross encoder

In [175]:
reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

In [176]:
pairs = []
for doc in retrieved_docs:
    pairs.append([query, doc.page_content])

In [177]:
pairs

[['how many employees are there in this company',
  '- **2019:** Rating: 4.0/5\n  *Strong start as Product Manager. Successfully transitioned from APM role with minimal friction.*\n\n## Compensation History\n- **2023:** Base Salary: $135,000 + Bonus: $20,000\n- **2022:** Base Salary: $128,000 + Bonus: $15,000\n- **2021:** Base Salary: $120,000 + Bonus: $18,000\n- **2020:** Base Salary: $110,000 + Bonus: $8,000\n- **2019:** Base Salary: $105,000 + Bonus: $10,000'],
 ['how many employees are there in this company',
  '- **2019:** Rating: 4.0/5\n  *Strong start as Product Manager. Successfully transitioned from APM role with minimal friction.*\n\n## Compensation History\n- **2023:** Base Salary: $135,000 + Bonus: $20,000\n- **2022:** Base Salary: $128,000 + Bonus: $15,000\n- **2021:** Base Salary: $120,000 + Bonus: $18,000\n- **2020:** Base Salary: $110,000 + Bonus: $8,000\n- **2019:** Base Salary: $105,000 + Bonus: $10,000'],
 ['how many employees are there in this company',
  '- **2019:

In [178]:
# Compute scores
# ---------------------------
scores = reranker.predict(pairs)

In [179]:
scores

array([-9.32739  , -9.32739  , -9.32739  , -9.32739  , -9.32739  ,
       -9.32739  , -9.32739  , -9.32739  , -9.327391 , -4.0131264],
      dtype=float32)

In [180]:
# Attach score to each document
# ---------------------------
scored_docs = []
for i in range(len(retrieved_docs)):
    scored_docs.append({
        "score": float(scores[i]),
        "doc": retrieved_docs[i]
    })

In [181]:
scored_docs

[{'score': -9.32738971710205,
  'doc': Document(id='e93460ab-c195-4464-986f-79c44c4acf31', metadata={'source': 'C:\\Users\\Mohamed Arshad\\Downloads\\My_RAG_Lab\\llm_engineering\\RAG\\knowledge-base\\employees\\Rachel Martinez.md'}, page_content='- **2019:** Rating: 4.0/5\n  *Strong start as Product Manager. Successfully transitioned from APM role with minimal friction.*\n\n## Compensation History\n- **2023:** Base Salary: $135,000 + Bonus: $20,000\n- **2022:** Base Salary: $128,000 + Bonus: $15,000\n- **2021:** Base Salary: $120,000 + Bonus: $18,000\n- **2020:** Base Salary: $110,000 + Bonus: $8,000\n- **2019:** Base Salary: $105,000 + Bonus: $10,000')},
 {'score': -9.32738971710205,
  'doc': Document(id='aeca4b05-309e-49f4-b2a7-56f0f5186af4', metadata={'source': 'C:\\Users\\Mohamed Arshad\\Downloads\\My_RAG_Lab\\llm_engineering\\RAG\\knowledge-base\\employees\\Rachel Martinez.md'}, page_content='- **2019:** Rating: 4.0/5\n  *Strong start as Product Manager. Successfully transitioned 

In [182]:
# Sort by score descending
# ---------------------------
scored_docs = sorted(scored_docs, key=lambda x: x["score"], reverse=True)


In [183]:
scored_docs

[{'score': -4.013126373291016,
  'doc': Document(id='ab09f0fd-02f0-41c9-aae2-e8570c968c74', metadata={'source': 'C:\\Users\\Mohamed Arshad\\Downloads\\My_RAG_Lab\\llm_engineering\\RAG\\knowledge-base\\employees\\Amanda Foster.md'}, page_content='# HR Record\n\n# Amanda Foster\n\n## Summary\n- **Date of Birth:** August 14, 1982\n- **Job Title:** HR Business Partner\n- **Location:** San Francisco, California\n- **Current Salary:** $115,000\n\n## Insurellm Career Progression\n- **September 2016 - Present:** HR Business Partner\n  - Partners with engineering and product leadership teams\n  - Supports 85 employees across multiple departments\n  - Leads talent acquisition strategy for technical roles\n  - Drives performance management and organizational development initiatives\n\n- **March 2013 - August 2016:** Senior HR Generalist at TechVentures Inc.\n  - Managed full-cycle HR operations for 150-person startup\n  - Led benefits administration and employee relations\n\n- **June 2009 - Febru

In [184]:
# Extract reranked docs
# ---------------------------
reranked_docs = [item["doc"] for item in scored_docs]

In [185]:
reranked_docs

[Document(id='ab09f0fd-02f0-41c9-aae2-e8570c968c74', metadata={'source': 'C:\\Users\\Mohamed Arshad\\Downloads\\My_RAG_Lab\\llm_engineering\\RAG\\knowledge-base\\employees\\Amanda Foster.md'}, page_content='# HR Record\n\n# Amanda Foster\n\n## Summary\n- **Date of Birth:** August 14, 1982\n- **Job Title:** HR Business Partner\n- **Location:** San Francisco, California\n- **Current Salary:** $115,000\n\n## Insurellm Career Progression\n- **September 2016 - Present:** HR Business Partner\n  - Partners with engineering and product leadership teams\n  - Supports 85 employees across multiple departments\n  - Leads talent acquisition strategy for technical roles\n  - Drives performance management and organizational development initiatives\n\n- **March 2013 - August 2016:** Senior HR Generalist at TechVentures Inc.\n  - Managed full-cycle HR operations for 150-person startup\n  - Led benefits administration and employee relations\n\n- **June 2009 - February 2013:** HR Coordinator at Enterpris

In [186]:
# Print reranked output
# ---------------------------
print("\nReranked Documents:")
for i, item in enumerate(scored_docs):
    doc = item["doc"]
    score = item["score"]
    print(f"Rank {i+1} | Score: {score:.4f} | Source: {doc.metadata.get('source','Unknown')}")
    print(doc.page_content)
    print("\n")


Reranked Documents:
Rank 1 | Score: -4.0131 | Source: C:\Users\Mohamed Arshad\Downloads\My_RAG_Lab\llm_engineering\RAG\knowledge-base\employees\Amanda Foster.md
# HR Record

# Amanda Foster

## Summary
- **Date of Birth:** August 14, 1982
- **Job Title:** HR Business Partner
- **Location:** San Francisco, California
- **Current Salary:** $115,000

## Insurellm Career Progression
- **September 2016 - Present:** HR Business Partner
  - Partners with engineering and product leadership teams
  - Supports 85 employees across multiple departments
  - Leads talent acquisition strategy for technical roles
  - Drives performance management and organizational development initiatives

- **March 2013 - August 2016:** Senior HR Generalist at TechVentures Inc.
  - Managed full-cycle HR operations for 150-person startup
  - Led benefits administration and employee relations

- **June 2009 - February 2013:** HR Coordinator at Enterprise Corp.
  - Supported recruiting and onboarding processes
  - Assi

In [187]:
# ---------------------------
# 1. Prepare context from top reranked docs
# ---------------------------
top_k = 3
context = "\n\n".join([doc.page_content for doc in reranked_docs[:top_k]])

# ---------------------------
# 2. Build final RAG prompt
# ---------------------------
final_prompt = f"""
You are an expert assistant. 
Answer the question using ONLY the information from the context.

Question:
{query}

Context:
{context}

If the answer is not found in the context, say:
"I don't know from the given context."
"""

# ---------------------------
# 3. Invoke the LLM for final answer
# ---------------------------
llm = ChatGroq(model="llama-3.1-8b-instant")
response = llm.invoke(final_prompt)

print("Final Answer:\n")
print(response.content)


Final Answer:

85 employees.
